INSTALAÇÃO DAS BIBLIOTECAS NECESSÁRIAS

In [ ]:
%pip install -U langchain-openai
%pip install -U langchain
%pip install -U qdrant-client
%pip install python-dotenv
%pip install -U langchain-qdrant

IMPORTAÇÃO DAS BIBLIOTECAS QUE SERÃO USADAS

In [ ]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance


from langchain_qdrant import Qdrant
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI, OpenAIEmbeddings

import os

from dotenv import load_dotenv
load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

CRIANDO UMA INSTÂNCIA DO QDRANT 

In [ ]:
client = QdrantClient(host="localhost", port=6333)

CRIAÇÃO DA MINHA COLEÇÃO DENTRO DO QDRANT

In [ ]:
client.create_collection(
    collection_name="teste-RAG",
    vectors_config=VectorParams(size=1536, distance=Distance.COSINE),
)

DEFINIÇÃO DO MODELO DE EMBEDDINGS E ASSOCIAÇÃO DO QDRANT COM O LANGCHAIN

In [ ]:
embeddings = OpenAIEmbeddings()

vectorstore = Qdrant(
    client=client,
    collection_name="teste-RAG",
    embeddings=embeddings,
    )

CRIAÇÃO DE UMA FUNÇÃO PARA DIVIDIR O TEXTO EM CHUNKS ("PEDAÇOS")

In [ ]:
def get_chunks(text):
    text_splitter = CharacterTextSplitter(
        separator="\n",
        chunk_size=1500,
        chunk_overlap=200,
        length_function=len
    )
    chunks = text_splitter.split_text(text)
    return chunks

with open('informacoe_escola_pos.csv', encoding='utf-8') as f:
    raw_text = f.read()

texts = get_chunks(raw_text)
vectorstore.add_texts(texts=texts)

DEFINIÇÃO DA PARTE FINAL DO RAG, PARTE DE PERGUNTAS E RESPOSTAS COM RECUPERAÇÃO DE DOCUMENTOS

In [ ]:
qa = RetrievalQA.from_chain_type(
    #Definição do modelo LLM a ser utilizado
    llm = OpenAI(),
    #Definição de como os textos recuperados serão enviados para o LLM
    chain_type="stuff",
    #Transforma o Qdrant, a base de dados, em algo capaz 
    #de buscar os textos mais relevantes
    retriever=vectorstore.as_retriever(),
)

TESTES PARA VER O FUNCIONAMENTO

In [ ]:
query = 'Quem são os professores?'
result = qa.invoke({"query": query})
print("Query: ", query)
print(result['result'])